In [1]:
import csv

### Questions1 and 2

In [2]:
with open('input_data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [(row['RECORD_CREATION_DATE'], int(row['LOAN_AMOUNT'])) for row in reader]

In [3]:
# get the max loan amount
max_loan_amount= max(data, key=lambda x: x[1])[1]

In [4]:
# get the preceding date
prev = data[0][0]
if prev[1] == max_loan_amount:
    print prev[0]
else:
    for row in data[1:]:
        if row[1] == max_loan_amount:
            print prev
            break
        else:
            prev = row[0]

26May2014:18:03:41.000000


### Questions 3 and 4

    3. If the input data were not sorted by RECORD_CREATION_DATE, how would you solve this problem?

    I would do it in the same way by i will sort it based on date and them i will follow the same procedre.
    Sorting is done based on first value in the tuple as below.

In [5]:
data = sorted(data, key= lambda x: x[0])

    4. Your friend suggests that csv.DictReader is a better choice than pandas.read_csv for dealing
    with huge datafiles. Why do you think your friend suggests this, and are they right?
    
    It depends upon what i am doing.
    
        If its for just reading the data and not much of operations on it, i would go with "csv".
    
        If data analysis should be performed on the read data and some sample plots are required for
        priliminary assesment then i would go with "pandas".
    

### Part 2

In [6]:
import re
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv("input_data.csv")
# Set to a pandas datetime
df["entry_recorded"] = pd.to_datetime(df['RECORD_CREATION_DATE'], format='%d%b%Y:%H:%M:%S.%f')
df.head(5)

,RECORD_CREATION_DATE,LOAN_AMOUNT,CREDIT_RANGE,EXISTING_DEBT,AGENT_NOTES,OFFICE_LOCATION,entry_recorded
0,01May2014:12:56:37.000000,2500,780-799,NaN,"Monthly gross income 1,250",WASHINGTON STATE,2014-05-01 12:56:37
1,01May2014:16:55:30.000000,4700,760-779,"$10,001-$20,000","Annual gross income: $15,000",NORTHERN CALIFORNIA,2014-05-01 16:55:30
2,01May2014:17:05:59.000000,5800,740-759,"$20,001-$30,000","Annual gross income: $15,000",WASHINGTON STATE,2014-05-01 17:05:59
3,02May2014:10:51:57.000000,2500,700-719,"$90,001-$100,000","Monthly gross income 5,480",SOUTHERN CALIFORNIA,2014-05-02 10:51:57
4,02May2014:14:19:03.000000,4600,780-799,"$20,001-$30,000",NaN,WASHINGTON STATE,2014-05-02 14:19:03


In [8]:
import re
# Returns monthly net income if it appears in the AGENT_NOTES field (divide annual income by 12)
# Field should be a float datatype
def monthly_income_parser(x):
    annual_income = 0
    month_income = 0
    month_match = False
    annual_match = False
    
    if x == np.nan:
        return np.nan
    
    if len(str(x)) > 3:
        # get monthly income
        if "Monthly gross income" in x:
            month_match = True

            # get monthly income and debt.
            if '...' in x:
                one, two = x.split('...')
                one = "".join(re.findall(r'[0-9]+', one))
                two = "".join(re.findall(r'[0-9]+', two))  

                one = float(one) if one != "" else  0
                two = float(one) if two != "" else  0

                # assuming debt is always less than monthly income
                month_income = '$' + str(max(one, two) - min(one, two))
            else:
                # no debt case
                print(x)
                one = "".join(re.findall(r'[0-9]+', x))
                one = int(one) if one != "" else  0
                month_income = '$' + str(one)

        elif "Annual gross income" in x:
            # annual income

            annual_match = True
            if '...' in x:
                one, two = x.split('...')
                one = "".join(re.findall(r'[0-9]+', one))
                two = "".join(re.findall(r'[0-9]+', two))

                one = int(one) if one != "" else  0
                two = int(two) if two != "" else  0

                if one >  two:
                    one = one / 12
                else:
                    two = two / 12

                annual_income = '$' + str(max(one, two) - min(one, two))
            else:
                one = "".join(re.findall(r'[0-9]+', x))
                one = int(one) if one != "" else  0
                annual_income = '$' + str(one)
            
    if month_match:
        return month_income
    elif annual_match:
        return annual_income
    else:
        return np.nan

In [9]:
df["monthly_income"] = df["AGENT_NOTES"].apply(monthly_income_parser)

Monthly gross income 1,250
Monthly gross income 5,480
Monthly gross income 2,570
Monthly gross income 14,750
Monthly gross income 7,500
Monthly gross income 2,650
Monthly gross income 2,900
Monthly gross income 8,210
Monthly gross income 2,190
Monthly gross income 1,250
Monthly gross income 4,380
Monthly gross income 8,780
Monthly gross income 2,940
Monthly gross income 13,410
Monthly gross income 3,230
Monthly gross income 16,670
Monthly gross income 15,860
Monthly gross income 4,620
Monthly gross income 5,900
Monthly gross income 5,740
Monthly gross income 8,200
Monthly gross income 15,360
Monthly gross income 8,430
Monthly gross income 4,650
Monthly gross income 2,230
Monthly gross income 1,250
Monthly gross income 2,160
Monthly gross income 4,730
Monthly gross income 6,220
Monthly gross income 1,250
Monthly gross income 14,890
Monthly gross income 2,450
Monthly gross income 1,250
Monthly gross income 1,470
Monthly gross income 2,090
Monthly gross income 1,800
Monthly gross income 1

In [10]:
def get_lower_bound(x):
    if '-' in x:
        return x.split('-')[0]
    elif '+' in x:
        return x.split('+')[0]
    
df["credit_score"] = df["CREDIT_RANGE"].apply(get_lower_bound)
print df.head(10)

        RECORD_CREATION_DATE  LOAN_AMOUNT CREDIT_RANGE     EXISTING_DEBT  \
0  01May2014:12:56:37.000000         2500      780-799               NaN   
1  01May2014:16:55:30.000000         4700      760-779   $10,001-$20,000   
2  01May2014:17:05:59.000000         5800      740-759   $20,001-$30,000   
3  02May2014:10:51:57.000000         2500      700-719  $90,001-$100,000   
4  02May2014:14:19:03.000000         4600      780-799   $20,001-$30,000   
5  02May2014:17:33:29.000000         8400      700-719        $1-$10,000   
6  02May2014:18:04:25.000000         7100      720-739               NaN   
7  02May2014:18:38:09.000000         2900         800+               NaN   
8  05May2014:12:51:52.000000         2500      760-779               NaN   
9  05May2014:14:43:47.000000         2700      740-759               NaN   

                                         AGENT_NOTES      OFFICE_LOCATION  \
0                         Monthly gross income 1,250     WASHINGTON STATE   
1        

In [11]:
def get_debt(x):
    if np.nan == x or str(x) == 'nan':
        return 0.0
    elif '-' in x:
        one = x.split('-')[0]
        one = "".join(re.findall(r'[0-9]+', one))
        return float(one)
    elif '+' in x:
        one = x.split('-')[0]
        one = "".join(re.findall(r'[0-9]+', one))
        return float(one)

df["debt"] = df["EXISTING_DEBT"].apply(get_debt)

df.head()

,RECORD_CREATION_DATE,LOAN_AMOUNT,CREDIT_RANGE,EXISTING_DEBT,AGENT_NOTES,OFFICE_LOCATION,entry_recorded,monthly_income,credit_score,debt
0,01May2014:12:56:37.000000,2500,780-799,NaN,"Monthly gross income 1,250",WASHINGTON STATE,2014-05-01 12:56:37,$1250,780,0.0
1,01May2014:16:55:30.000000,4700,760-779,"$10,001-$20,000","Annual gross income: $15,000",NORTHERN CALIFORNIA,2014-05-01 16:55:30,$15000,760,10001.0
2,01May2014:17:05:59.000000,5800,740-759,"$20,001-$30,000","Annual gross income: $15,000",WASHINGTON STATE,2014-05-01 17:05:59,$15000,740,20001.0
3,02May2014:10:51:57.000000,2500,700-719,"$90,001-$100,000","Monthly gross income 5,480",SOUTHERN CALIFORNIA,2014-05-02 10:51:57,$5480,700,90001.0
4,02May2014:14:19:03.000000,4600,780-799,"$20,001-$30,000",NaN,WASHINGTON STATE,2014-05-02 14:19:03,NaN,780,20001.0


In [12]:
grouped_df = df.groupby([df['OFFICE_LOCATION'], df['CREDIT_RANGE']])
print grouped_df.count()
print grouped_df.mean()

                                  RECORD_CREATION_DATE  LOAN_AMOUNT  \
OFFICE_LOCATION     CREDIT_RANGE                                      
ARIZONA             700-719                        322          322   
                    720-739                        102          102   
                    740-759                        162          162   
                    760-779                        140          140   
                    780-799                        149          149   
                    800+                           144          144   
NEVADA              700-719                        227          227   
                    720-739                         51           51   
                    740-759                        104          104   
                    760-779                         83           83   
                    780-799                         83           83   
                    800+                           106          106   
NORTHE

    14. What is the advantage of using built-in pandas string methods as opposed to apply()?
    
    1. Easy to operate on the dataframe.
    2. Comes with string function like split, contains
    3. Apply is generally based on user wriiten function so may not be efficient.


    15. What is the average number of hours between record creations? Please show the code you wrote.

In [13]:
l = len(df['entry_recorded']) - 1
for i in xrange(l):
    df['time_diff'] = df.entry_recorded[i+1] - df.entry_recorded[i]

total_time = df['time_diff'].sum()
avg_time = total_time / (l + 1)
print "Average hours {avg_time}." .format(avg_time=avg_time)

Average hours 0 days 02:24:53.


In [14]:
grouped_df_california = df.groupby([df['OFFICE_LOCATION']])

total_records = (
    len(grouped_df_california.get_group('NORTHERN CALIFORNIA')) + 
    len(grouped_df_california.get_group('SOUTHERN CALIFORNIA')))

print "Total California records: {total_records}." .format(total_records=total_records)

Total California records: 2405.
